# LSTM 

## Outline
1. [Imports](#imports)
2. [Data](#data)
3. [Model](#model)
5. [Train](#train)
6. [Predict](#predict)

<a name=imports></a>
## Imports

In [1]:
import os
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

<a name=data></a>
## Data

### Ingestion 

In [2]:
# Get the path of the current working directory
curPath = os.getcwd()
# Appened the parent directory to the current path to step out of the current folder
parentDir = os.path.abspath(os.path.join(curPath, os.pardir))
print("Parent Directory", parentDir)
# Save the path to all of the datasets
india_cases_path = os.path.join(parentDir, "cleaned_datasets/india/daily_cases_india.csv")
india_vacc_path = os.path.join(parentDir, "cleaned_datasets/india/daily_vacc_india.csv")
usa_cases_path = os.path.join(parentDir, "cleaned_datasets/usa/daily_cases_usa.csv")
usa_vacc_path = os.path.join(parentDir, "cleaned_datasets/usa/vacc_usa.csv")

# Quick check to make sure the path exists
print("Path:", india_cases_path)
print("Exists:", os.path.exists(india_cases_path))

Parent Directory /deep-co-training
Path: /deep-co-training/cleaned_datasets/india/daily_cases_india.csv
Exists: True


In [3]:
# Load the data as a pandas dataframe
india_cases_df = pd.read_csv(india_cases_path)
india_vacc_df =  pd.read_csv(india_vacc_path)

usa_cases_df = pd.read_csv(usa_cases_path)
usa_vacc_df = pd.read_csv(usa_vacc_path)

# Visualize the datasets
print('India Cases:\n',india_cases_df.head(),'\n')
print('India Vacc:\n',india_vacc_df.head(),'\n')

print('USA Cases:\n',usa_cases_df.head(),'\n')
print('USA Vacc:\n',usa_vacc_df.head(),'\n')

India Cases:
    Unnamed: 0        Date  Confirmed  Deaths  Recovered  Active
0           0  2020-01-30        1.0     0.0        0.0     0.0
1           1  2020-01-31        0.0     0.0        0.0     0.0
2           2  2020-02-01        0.0     0.0        0.0     0.0
3           3  2020-02-02        1.0     0.0        0.0     0.0
4           4  2020-02-03        1.0     0.0        0.0     0.0 

India Vacc:
    Updated On  Total_Doses  First_Dose  Second_Dose
0  2021-01-16          NaN         NaN          NaN
1  2021-01-17      20656.0     20656.0          0.0
2  2021-01-18      81690.0     81690.0          0.0
3  2021-01-19     192152.0    192152.0          0.0
4  2021-01-20     111510.0    111510.0          0.0 

USA Cases:
          Date  Confirmed  Deaths  Recovered
0  2020-04-12        NaN     NaN        NaN
1  2020-04-13    25322.0  1546.0    11785.0
2  2020-04-14    26713.0  2305.0     6484.0
3  2020-04-15    29380.0  2478.0     6093.0
4  2020-04-16    31542.0  4616.0     5234

### Pre-Processing

In [4]:
# Select only the Confirmed column for univariate analysis
# Selecting from the first index because the 0th index is NaN
india_cases_df = india_cases_df[["Confirmed"]][1:]
usa_cases_df = usa_cases_df[["Confirmed"]][1:]

# Visualize the datasets
print('India Cases:\n',india_cases_df.head(),'\n')
print('USA Cases:\n',usa_cases_df.head(),'\n')

India Cases:
    Confirmed
1        0.0
2        0.0
3        1.0
4        1.0
5        0.0 

USA Cases:
    Confirmed
1    25322.0
2    26713.0
3    29380.0
4    31542.0
5    32022.0 



In [5]:
# Normalize the data
india_cases_mean = india_cases_df.mean()
india_cases_std = india_cases_df.std()

usa_cases_mean = usa_cases_df.mean()
usa_cases_std = usa_cases_df.std()


india_cases_df = (india_cases_df-india_cases_mean)/india_cases_std
usa_cases_df = (usa_cases_df-usa_cases_mean)/usa_cases_std

# Visualize the datasets
print('India Cases:\n',india_cases_df.head(),'\n')
print('USA Cases:\n',usa_cases_df.head(),'\n')

India Cases:
    Confirmed
1  -0.687995
2  -0.687995
3  -0.687983
4  -0.687983
5  -0.687995 

USA Cases:
    Confirmed
1  -0.817861
2  -0.797170
3  -0.757499
4  -0.725340
5  -0.718200 



In [6]:
# Create train test splits
india_cases_train, india_cases_test = train_test_split(india_cases_df, test_size=0.2)
india_vacc_train, india_vacc_test = train_test_split(india_vacc_df, test_size=0.2)

usa_cases_train, usa_cases_test = train_test_split(usa_cases_df, test_size=0.2)
usa_vacc_train, usa_vacc_test = train_test_split(usa_vacc_df, test_size=0.2)

# Visualize splits
print('India Cases:\n',india_cases_train,'\n')
print('USA Cases:\n',usa_cases_train,'\n')

India Cases:
      Confirmed
249   0.081848
135  -0.544021
68   -0.681298
433   0.905157
427   0.335657
..         ...
601  -0.286870
552  -0.147910
370  -0.525914
452   3.370913
491   0.826498

[496 rows x 1 columns] 

USA Cases:
      Confirmed
146  -0.542292
507   1.810341
209   0.764469
39   -0.815258
69   -0.702091
..         ...
258   2.055507
445  -0.979431
249   2.338424
162  -0.439359
87   -0.314009

[438 rows x 1 columns] 



In [7]:
# Convert to numpy array
india_cases_train, india_cases_test = india_cases_train.to_numpy().flatten(), india_cases_test.to_numpy()
usa_cases_train, usa_cases_test = usa_cases_train.to_numpy().flatten(), usa_cases_test.to_numpy()

# Visualize outputs
print('India Cases:\n',india_cases_train,'\n')
print('USA Cases:\n',usa_cases_train,'\n')

India Cases:
 [ 8.18483055e-02 -5.44020939e-01 -6.81297767e-01  9.05157440e-01
  3.35656510e-01 -5.99646258e-02  1.08285875e-01 -4.83041128e-01
  7.39840137e-03  4.75622330e-01 -4.26647681e-01 -6.87530198e-01
 -6.87944855e-01 -2.47564180e-02  4.05419526e-01 -3.72955793e-01
  3.27715187e-01 -4.52620331e-01 -5.51484778e-01 -6.87743809e-01
  2.84772821e+00 -2.27448638e-01 -1.26523468e-01 -5.17281801e-01
 -3.77479331e-01 -4.80616008e-01 -4.06140973e-01  3.23166518e-01
 -1.37053261e-01 -1.60060480e-01  3.74747258e+00 -3.09864996e-01
  9.78189094e-02 -1.98585949e-01 -5.25612651e-01 -1.62636384e-01
 -6.77339671e-01 -6.26877086e-01 -2.48520788e-01  5.53552846e-01
  2.41114556e-01 -3.59850096e-01 -6.64208844e-01 -6.87982552e-01
 -3.71774646e-01 -3.54937031e-01 -5.26140397e-01  2.59652104e+00
 -6.76409832e-01 -1.87943069e-01 -6.87969986e-01 -6.66546005e-01
 -3.80243716e-01  3.66950437e+00 -9.41299067e-02 -6.50763882e-01
 -6.42923082e-01 -4.87351055e-01 -1.93346184e-01 -1.11206265e-01
  4.3632015

In [8]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [9]:
# choose a number of time steps
n_steps = 3
# split into samples
india_cases_train_X, india_cases_train_y = split_sequence(india_cases_train, n_steps)
india_cases_test_X, india_cases_test_y = split_sequence(india_cases_test, n_steps)


# summarize the data
for i in range(0,5):
    print(india_cases_train_X[i], india_cases_train_y[i])

[ 0.08184831 -0.54402094 -0.68129777] 0.9051574397524952
[-0.54402094 -0.68129777  0.90515744] 0.3356565104389887
[-0.68129777  0.90515744  0.33565651] -0.05996462583698051
[ 0.90515744  0.33565651 -0.05996463] 0.10828587486923602
[ 0.33565651 -0.05996463  0.10828587] -0.48304112761280654


In [10]:
# Reshape the data into [samples, timesteps, features]
# univariate
n_features = 1

india_cases_train_X = india_cases_train_X.reshape((india_cases_train_X.shape[0], 
                                                   india_cases_train_X.shape[1], n_features))
india_cases_test_X = india_cases_test_X.reshape((india_cases_test_X.shape[0], 
                                                 india_cases_test_X.shape[1], n_features))

# Visualize outputs
print('India Cases:\n',india_cases_train_X,'\n')

India Cases:
 [[[ 0.08184831]
  [-0.54402094]
  [-0.68129777]]

 [[-0.54402094]
  [-0.68129777]
  [ 0.90515744]]

 [[-0.68129777]
  [ 0.90515744]
  [ 0.33565651]]

 ...

 [[-0.47767571]
  [-0.28687034]
  [-0.14790975]]

 [[-0.28687034]
  [-0.14790975]
  [-0.52591422]]

 [[-0.14790975]
  [-0.52591422]
  [ 3.37091313]]] 



<a name=model></a>
## Model 

### Univariate non-stacked

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.losses import MeanAbsolutePercentageError, MeanAbsoluteError

In [68]:
# Define the model
model_uni = Sequential()
model_uni.add(LSTM(150, activation='relu', input_shape=(n_steps, n_features)))
model_uni.add(Dense(1))
model_uni.compile(optimizer='adam',loss=MeanAbsoluteError())
model_uni.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_31 (LSTM)               (None, 150)               91200     
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 151       
Total params: 91,351
Trainable params: 91,351
Non-trainable params: 0
_________________________________________________________________


### Univariate stacked 

In [70]:
# Define the model
model_uni_stacked = Sequential()
model_uni_stacked.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model_uni_stacked.add(LSTM(150, activation='relu'))
model_uni_stacked.add(Dense(1))
model_uni_stacked.compile(optimizer='adam',loss=MeanAbsoluteError())
model_uni_stacked.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 3, 100)            40800     
_________________________________________________________________
lstm_33 (LSTM)               (None, 150)               150600    
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 151       
Total params: 191,551
Trainable params: 191,551
Non-trainable params: 0
_________________________________________________________________


<a name=train></a>
## Train 

In [66]:
# Tensorboard callback
logdir = os.path.join(parentDir+"/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

### Univariate non-stacked

In [69]:
# Fit the model|
model_uni.fit(india_cases_train_X,
          india_cases_train_y, 
          validation_data=(india_cases_test_X, india_cases_test_y),
          epochs=500,
          callbacks=[tensorboard_callback])

Epoch 1/500
16/16 [==============================] - 1s 28ms/step - loss: 0.5644 - val_loss: 0.5934
Epoch 2/500
16/16 [==============================] - 0s 6ms/step - loss: 0.5322 - val_loss: 0.5941
Epoch 3/500
16/16 [==============================] - 0s 6ms/step - loss: 0.5243 - val_loss: 0.5954
Epoch 4/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5229 - val_loss: 0.5970
Epoch 5/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5223 - val_loss: 0.5971
Epoch 6/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5216 - val_loss: 0.5946
Epoch 7/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5217 - val_loss: 0.5968
Epoch 8/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5211 - val_loss: 0.5966
Epoch 9/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5193 - val_loss: 0.5968
Epoch 10/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5197 - val_loss: 0.5956
Epoch 11

16/16 [==============================] - 0s 6ms/step - loss: 0.5085 - val_loss: 0.5988
Epoch 84/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5109 - val_loss: 0.5972
Epoch 85/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5089 - val_loss: 0.5985
Epoch 86/500
16/16 [==============================] - 0s 6ms/step - loss: 0.5090 - val_loss: 0.5967
Epoch 87/500
16/16 [==============================] - 0s 6ms/step - loss: 0.5092 - val_loss: 0.5967
Epoch 88/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5087 - val_loss: 0.5982
Epoch 89/500
16/16 [==============================] - 0s 6ms/step - loss: 0.5089 - val_loss: 0.5973
Epoch 90/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5087 - val_loss: 0.5965
Epoch 91/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5105 - val_loss: 0.5983
Epoch 92/500
16/16 [==============================] - 0s 6ms/step - loss: 0.5114 - val_loss: 0.5974
Epoch 93/500


16/16 [==============================] - 0s 6ms/step - loss: 0.5065 - val_loss: 0.6008
Epoch 165/500
16/16 [==============================] - 0s 6ms/step - loss: 0.5066 - val_loss: 0.6007
Epoch 166/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5062 - val_loss: 0.6007
Epoch 167/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5051 - val_loss: 0.5999
Epoch 168/500
16/16 [==============================] - 0s 6ms/step - loss: 0.5040 - val_loss: 0.6022
Epoch 169/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5032 - val_loss: 0.6014
Epoch 170/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5039 - val_loss: 0.6015
Epoch 171/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5041 - val_loss: 0.6014
Epoch 172/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5045 - val_loss: 0.6026
Epoch 173/500
16/16 [==============================] - 0s 5ms/step - loss: 0.5045 - val_loss: 0.6039
Epoc

16/16 [==============================] - 0s 5ms/step - loss: 0.4988 - val_loss: 0.6098
Epoch 246/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4976 - val_loss: 0.6063
Epoch 247/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4978 - val_loss: 0.6068
Epoch 248/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4967 - val_loss: 0.6087
Epoch 249/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4986 - val_loss: 0.6051
Epoch 250/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4976 - val_loss: 0.6075
Epoch 251/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4973 - val_loss: 0.6081
Epoch 252/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4985 - val_loss: 0.6070
Epoch 253/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4968 - val_loss: 0.6073
Epoch 254/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4988 - val_loss: 0.6042
Epoc

16/16 [==============================] - 0s 5ms/step - loss: 0.4930 - val_loss: 0.6097
Epoch 327/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4939 - val_loss: 0.6085
Epoch 328/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4931 - val_loss: 0.6108
Epoch 329/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4926 - val_loss: 0.6100
Epoch 330/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4927 - val_loss: 0.6091
Epoch 331/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4916 - val_loss: 0.6107
Epoch 332/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4920 - val_loss: 0.6093
Epoch 333/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4934 - val_loss: 0.6121
Epoch 334/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4944 - val_loss: 0.6097
Epoch 335/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4925 - val_loss: 0.6100
Epoc

16/16 [==============================] - 0s 6ms/step - loss: 0.4874 - val_loss: 0.6117
Epoch 408/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4869 - val_loss: 0.6137
Epoch 409/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4880 - val_loss: 0.6088
Epoch 410/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4894 - val_loss: 0.6123
Epoch 411/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4903 - val_loss: 0.6107
Epoch 412/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4891 - val_loss: 0.6113
Epoch 413/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4885 - val_loss: 0.6108
Epoch 414/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4899 - val_loss: 0.6102
Epoch 415/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4921 - val_loss: 0.6107
Epoch 416/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4922 - val_loss: 0.6103
Epoc

16/16 [==============================] - 0s 6ms/step - loss: 0.4834 - val_loss: 0.6141
Epoch 489/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4833 - val_loss: 0.6156
Epoch 490/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4829 - val_loss: 0.6179
Epoch 491/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4833 - val_loss: 0.6151
Epoch 492/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4838 - val_loss: 0.6164
Epoch 493/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4828 - val_loss: 0.6158
Epoch 494/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4836 - val_loss: 0.6177
Epoch 495/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4855 - val_loss: 0.6142
Epoch 496/500
16/16 [==============================] - 0s 5ms/step - loss: 0.4838 - val_loss: 0.6170
Epoch 497/500
16/16 [==============================] - 0s 7ms/step - loss: 0.4850 - val_loss: 0.6221
Epoc

### Univariate stacked 

In [71]:
# Fit the model
model_uni_stacked.fit(india_cases_train_X,
          india_cases_train_y, 
          validation_data=(india_cases_test_X, india_cases_test_y),
          epochs=500,
          callbacks=[tensorboard_callback])

Epoch 1/500
16/16 [==============================] - 2s 41ms/step - loss: 0.5688 - val_loss: 0.5914
Epoch 2/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5349 - val_loss: 0.5939
Epoch 3/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5274 - val_loss: 0.5932
Epoch 4/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5249 - val_loss: 0.5937
Epoch 5/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5243 - val_loss: 0.5936
Epoch 6/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5241 - val_loss: 0.5935
Epoch 7/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5256 - val_loss: 0.5945
Epoch 8/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5210 - val_loss: 0.5937
Epoch 9/500
16/16 [==============================] - 0s 8ms/step - loss: 0.5212 - val_loss: 0.5935
Epoch 10/500
16/16 [==============================] - 0s 8ms/step - loss: 0.5212 - val_loss: 0.5937
Epoch 11

Epoch 83/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5138 - val_loss: 0.5943
Epoch 84/500
16/16 [==============================] - 0s 8ms/step - loss: 0.5126 - val_loss: 0.5970
Epoch 85/500
16/16 [==============================] - 0s 10ms/step - loss: 0.5107 - val_loss: 0.5938
Epoch 86/500
16/16 [==============================] - 0s 10ms/step - loss: 0.5107 - val_loss: 0.5938
Epoch 87/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5103 - val_loss: 0.5943
Epoch 88/500
16/16 [==============================] - 0s 10ms/step - loss: 0.5101 - val_loss: 0.5970
Epoch 89/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5110 - val_loss: 0.5953
Epoch 90/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5108 - val_loss: 0.5942
Epoch 91/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5107 - val_loss: 0.5970
Epoch 92/500
16/16 [==============================] - 0s 10ms/step - loss: 0.5116 - val_loss: 0.5

16/16 [==============================] - 0s 9ms/step - loss: 0.5009 - val_loss: 0.6090
Epoch 165/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5010 - val_loss: 0.6085
Epoch 166/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5035 - val_loss: 0.6113
Epoch 167/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5002 - val_loss: 0.6088
Epoch 168/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5000 - val_loss: 0.6097
Epoch 169/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5013 - val_loss: 0.6048
Epoch 170/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5031 - val_loss: 0.6102
Epoch 171/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5015 - val_loss: 0.6110
Epoch 172/500
16/16 [==============================] - 0s 10ms/step - loss: 0.4997 - val_loss: 0.6122
Epoch 173/500
16/16 [==============================] - 0s 9ms/step - loss: 0.5001 - val_loss: 0.6100
Epo

16/16 [==============================] - 0s 9ms/step - loss: 0.4918 - val_loss: 0.6242
Epoch 246/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4944 - val_loss: 0.6174
Epoch 247/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4930 - val_loss: 0.6224
Epoch 248/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4968 - val_loss: 0.6187
Epoch 249/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4944 - val_loss: 0.6172
Epoch 250/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4955 - val_loss: 0.6260
Epoch 251/500
16/16 [==============================] - 0s 10ms/step - loss: 0.4950 - val_loss: 0.6151
Epoch 252/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4946 - val_loss: 0.6248
Epoch 253/500
16/16 [==============================] - 0s 8ms/step - loss: 0.4920 - val_loss: 0.6190
Epoch 254/500
16/16 [==============================] - 0s 10ms/step - loss: 0.4919 - val_loss: 0.6193
Ep

16/16 [==============================] - 0s 9ms/step - loss: 0.4896 - val_loss: 0.6230
Epoch 327/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4877 - val_loss: 0.6242
Epoch 328/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4882 - val_loss: 0.6175
Epoch 329/500
16/16 [==============================] - 0s 10ms/step - loss: 0.4871 - val_loss: 0.6252
Epoch 330/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4875 - val_loss: 0.6189
Epoch 331/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4878 - val_loss: 0.6235
Epoch 332/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4891 - val_loss: 0.6221
Epoch 333/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4877 - val_loss: 0.6210
Epoch 334/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4883 - val_loss: 0.6186
Epoch 335/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4888 - val_loss: 0.6210
Epo

16/16 [==============================] - 0s 9ms/step - loss: 0.4798 - val_loss: 0.6253
Epoch 408/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4809 - val_loss: 0.6332
Epoch 409/500
16/16 [==============================] - 0s 10ms/step - loss: 0.4845 - val_loss: 0.6341
Epoch 410/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4815 - val_loss: 0.6286
Epoch 411/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4798 - val_loss: 0.6359
Epoch 412/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4802 - val_loss: 0.6241
Epoch 413/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4799 - val_loss: 0.6365
Epoch 414/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4785 - val_loss: 0.6314
Epoch 415/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4792 - val_loss: 0.6346
Epoch 416/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4817 - val_loss: 0.6345
Epo

16/16 [==============================] - 0s 9ms/step - loss: 0.4708 - val_loss: 0.6479
Epoch 489/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4671 - val_loss: 0.6522
Epoch 490/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4659 - val_loss: 0.6564
Epoch 491/500
16/16 [==============================] - 0s 8ms/step - loss: 0.4626 - val_loss: 0.6546
Epoch 492/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4644 - val_loss: 0.6527
Epoch 493/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4671 - val_loss: 0.6511
Epoch 494/500
16/16 [==============================] - 0s 8ms/step - loss: 0.4650 - val_loss: 0.6452
Epoch 495/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4646 - val_loss: 0.6517
Epoch 496/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4660 - val_loss: 0.6612
Epoch 497/500
16/16 [==============================] - 0s 9ms/step - loss: 0.4666 - val_loss: 0.6447
Epoc

<a name=predict></a>
## Predict

### Univariate

In [72]:
x_input = np.array(india_cases_test_X)
print(x_input)
yhat = model_uni.predict(x_input)
print(yhat)

[[[ 1.83436764]
  [-0.43624764]
  [-0.10223458]]

 [[-0.43624764]
  [-0.10223458]
  [-0.47536368]]

 [[-0.10223458]
  [-0.47536368]
  [-0.21208117]]

 [[-0.47536368]
  [-0.21208117]
  [-0.52361475]]

 [[-0.21208117]
  [-0.52361475]
  [ 0.45296694]]

 [[-0.52361475]
  [ 0.45296694]
  [ 3.62374124]]

 [[ 0.45296694]
  [ 3.62374124]
  [ 0.39328136]]

 [[ 3.62374124]
  [ 0.39328136]
  [-0.12703865]]

 [[ 0.39328136]
  [-0.12703865]
  [ 2.10547838]]

 [[-0.12703865]
  [ 2.10547838]
  [-0.15062388]]

 [[ 2.10547838]
  [-0.15062388]
  [-0.45088631]]

 [[-0.15062388]
  [-0.45088631]
  [-0.18882265]]

 [[-0.45088631]
  [-0.18882265]
  [ 0.1233769 ]]

 [[-0.18882265]
  [ 0.1233769 ]
  [ 3.22197563]]

 [[ 0.1233769 ]
  [ 3.22197563]
  [-0.68799512]]

 [[ 3.22197563]
  [-0.68799512]
  [-0.38197774]]

 [[-0.68799512]
  [-0.38197774]
  [-0.29698548]]

 [[-0.38197774]
  [-0.29698548]
  [ 0.1100199 ]]

 [[-0.29698548]
  [ 0.1100199 ]
  [ 0.43194505]]

 [[ 0.1100199 ]
  [ 0.43194505]
  [-0.13437683]]



### Univariate stacked 

In [73]:
x_input = np.array(india_cases_test_X)
print(x_input)
yhat_stacked = model_uni_stacked.predict(x_input)
print(yhat)

[[[ 1.83436764]
  [-0.43624764]
  [-0.10223458]]

 [[-0.43624764]
  [-0.10223458]
  [-0.47536368]]

 [[-0.10223458]
  [-0.47536368]
  [-0.21208117]]

 [[-0.47536368]
  [-0.21208117]
  [-0.52361475]]

 [[-0.21208117]
  [-0.52361475]
  [ 0.45296694]]

 [[-0.52361475]
  [ 0.45296694]
  [ 3.62374124]]

 [[ 0.45296694]
  [ 3.62374124]
  [ 0.39328136]]

 [[ 3.62374124]
  [ 0.39328136]
  [-0.12703865]]

 [[ 0.39328136]
  [-0.12703865]
  [ 2.10547838]]

 [[-0.12703865]
  [ 2.10547838]
  [-0.15062388]]

 [[ 2.10547838]
  [-0.15062388]
  [-0.45088631]]

 [[-0.15062388]
  [-0.45088631]
  [-0.18882265]]

 [[-0.45088631]
  [-0.18882265]
  [ 0.1233769 ]]

 [[-0.18882265]
  [ 0.1233769 ]
  [ 3.22197563]]

 [[ 0.1233769 ]
  [ 3.22197563]
  [-0.68799512]]

 [[ 3.22197563]
  [-0.68799512]
  [-0.38197774]]

 [[-0.68799512]
  [-0.38197774]
  [-0.29698548]]

 [[-0.38197774]
  [-0.29698548]
  [ 0.1100199 ]]

 [[-0.29698548]
  [ 0.1100199 ]
  [ 0.43194505]]

 [[ 0.1100199 ]
  [ 0.43194505]
  [-0.13437683]]



<a name=evaluate></a>
## Evaluate

### Univariate 

In [74]:
mape = MeanAbsolutePercentageError()
mape(india_cases_test_y, yhat).numpy()

129.73888

### Univariate stacked 

In [75]:
mape(india_cases_test_y, yhat_stacked).numpy()

130.9539